In [ ]:
import numpy as np
import mikeio

In [ ]:
ds_surge = mikeio.read("../tests/testdata/uv_vertical_daily.dfs2")
ds_surge

<mikeio.Dataset>
dims: (time:3, y:5, x:11)
time: 2021-08-14 12:00:00 - 2021-08-16 12:00:00 (3 records)
geometry: Grid2D (ny=5, nx=11)
items:
 0: eastward_sea_water_velocity <u velocity component> (meter per sec)
 1: northward_sea_water_velocity <v velocity component> (meter per sec)

In [ ]:
ds_tide = mikeio.read("../tests/testdata/vu_tide_hourly.dfs1")
ds_tide

<mikeio.Dataset>
dims: (time:721, x:11)
time: 2021-08-01 00:00:00 - 2021-08-31 00:00:00 (721 records)
geometry: Grid1D (n=11, dx=0.09818)
items:
 0: Tidal current component (geographic North) <v velocity component> (meter per sec)
 1: Tidal current component (geographic East) <u velocity component> (meter per sec)

In [ ]:
ds_surge_h = ds_surge.interp_time(ds_tide)
ds_surge_h

<mikeio.Dataset>
dims: (time:721, y:5, x:11)
time: 2021-08-01 00:00:00 - 2021-08-31 00:00:00 (721 records)
geometry: Grid2D (ny=5, nx=11)
items:
 0: eastward_sea_water_velocity <u velocity component> (meter per sec)
 1: northward_sea_water_velocity <v velocity component> (meter per sec)

In [ ]:
u_tide = ds_tide["Tidal current component (geographic East)"].to_numpy()
u_tide = np.expand_dims(u_tide, 1)
u_tide.shape

In [ ]:
u_surge = ds_surge_h["eastward_sea_water_velocity"]
u_combined = u_surge + u_tide
u_combined.item = mikeio.ItemInfo("U", mikeio.EUMType.u_velocity_component)

In [ ]:
v_tide = ds_tide["Tidal current component (geographic North)"].to_numpy()
v_tide = np.expand_dims(v_tide, 1)
v_surge = ds_surge_h["northward_sea_water_velocity"]
v_combined = v_surge + v_tide
v_combined.item = mikeio.ItemInfo("V", mikeio.EUMType.u_velocity_component)

In [ ]:
ds_combined = mikeio.Dataset([u_combined, v_combined])

In [ ]:
ds_combined_1 = ds_combined.dropna()
ds_combined_1

<mikeio.Dataset>
dims: (time:49, y:5, x:11)
time: 2021-08-14 12:00:00 - 2021-08-16 12:00:00 (49 records)
geometry: Grid2D (ny=5, nx=11)
items:
 0: U <u velocity component> (meter per sec)
 1: V <u velocity component> (meter per sec)

In [ ]:
ds_combined_1.to_dfs('uv_combined.dfs2')

In [ ]:
import os
os.remove("uv_combined.dfs2")

In [ ]:
import marimo as mo

# Boundary

The task is to combine current velocities from an oceanographic model without tides with tidal current.

* The oceanographic model data is vertically resolved and available in a vertical transect as a Dfs2 with daily timestep
* The tidal model is vertically integrated and available in a line transect as a Dfs1 with hourly timestep
* The spatial grid is identical

## Steps
1. Read data
2. Interpolate in time
3. Calculate $\mathbf{U}_{combined} = \mathbf{U}_{tide} + \mathbf{U}_{residual}$
4. Write to new dfs2

### 1. Interpolate in time

### 2. Combine

**Note that the naming and order is inconsistent between the two data sources!**

### 3. Write to dfs2

# Clean up